<a href="https://colab.research.google.com/github/yz2873/Unsupervised_Learning/blob/master/Langchain_embeddings_pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -r requirements.txt -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.1/179.1 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.0/276.0 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip show langchain

Name: langchain
Version: 0.0.291
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, langsmith, numexpr, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


## **Load API Keys**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
from dotenv import load_dotenv, find_dotenv

In [ ]:
load_dotenv('/content/drive/My Drive/.env', override=True)

True

## **Splitting and Embedding Text Using Langchain**

In [ ]:
#Upload the Churchill.txt file
!ls Churchill.txt

Churchill.txt


In [ ]:
# Read it into churchill_speech
with open('Churchill.txt') as fs:
  churchill_speech = fs.read()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 100,
    chunk_overlap = 20,
    length_function = len
)

In [ ]:
chunks = text_splitter.create_documents([churchill_speech])

In [ ]:
print(chunks[0])

page_content='From the moment that the French defenses at Sedan and on the Meuse were broken at the end of the' metadata={}


In [ ]:
print(chunks[1])

page_content='at the end of the second week of May, only a rapid retreat to Amiens and the south could have saved' metadata={}


In [ ]:
print(f"Length of chunks {len(chunks)}")

Length of chunks 269


In [ ]:
def print_embedding_cost(texts):
  import tiktoken
  enc = tiktoken.encoding_for_model('text-embedding-ada-002')
  total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
  print(f"Total Tokens: {total_tokens}")
  print(f"Embedding Cost in USD: {total_tokens/1000*0.0004:.6f}")

In [ ]:
print_embedding_cost(chunks)

Total Tokens: 5389
Embedding Cost in USD: 0.002156


## **Embedding Text**

In [ ]:
from langchain.embeddings import OpenAIEmbeddings

In [ ]:
embeddings = OpenAIEmbeddings()

In [ ]:
vector = embedding.embed_query(chunks[0].page_content)

In [ ]:
print(vector)

[-0.00992929449021495, 0.00030837638124161246, -7.964618104593282e-05, 0.01523081647033575, -0.010698321051599248, 0.039744398656501194, -0.0254527492711791, -0.04124161887316259, 0.011140681570158297, -0.0309788540975346, 0.017639979351079194, 0.026528026353657603, 0.011943736795062, -0.0244999727696673, 0.0049816611112059, 0.008452490726607151, -0.0016120641620717248, 0.024105250331125795, 0.006393812463036451, -0.030488855498958002, 0.0033074963611579877, -0.027004414604413503, -0.003838329029994975, 0.012297625396173751, 0.01122915441892815, -0.0033772531190292745, 0.005349160525799624, -0.044317728843990004, -0.016169981692704297, -0.002802184258637999, 0.00036877561753255304, -0.0184022016698757, -0.0192869227069938, -0.022553584997891396, -0.013400124105616201, -0.014359705781038347, 0.0017166994152939747, -0.011140681570158297, 0.013311651256846347, -0.003401072624699325, 0.029263855277226296, -0.01486331659008075, 0.005563535611168801, 0.013304846082935999, -0.0253983041546060

## **Inserts the Embeddings into a Pinecone Index**

In [ ]:
import os
import pinecone
from langchain.vectorstores import Pinecone

/usr/local/lib/python3.10/dist-packages/pinecone/index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [ ]:
pinecone.init(
    api_key=os.environ.get('PINECONE_API_KEY'),
    environment=os.environ.get('PINECONE_ENV')
)

## **We will remove all indexes and re-create them**

In [ ]:
indexes = pinecone.list_indexes()

In [ ]:
len(indexes)

1

In [ ]:
print(indexes[0])

langchain-pinecone


In [ ]:
pinecone.delete_index(indexes[0])

In [ ]:
pinecone.list_indexes()

[]

## **Create new index named 'churchill-speech'**

In [ ]:
index_name = 'churchill-speech'
pinecone.create_index(index_name, dimension=1536, metric='cosine')

In [ ]:
pinecone.list_indexes()

['churchill-speech']

In [ ]:
vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)

## **Ask the question (Similarity Search)**

In [ ]:
query = 'Where should we fight?'
result = vector_store.similarity_search(query)

In [ ]:
print(result)

[Document(page_content='on the beaches, we shall fight on the landing grounds, we shall fight in the fields and in the', metadata={}), Document(page_content='fields and in the streets, we shall fight in the hills; we shall never surrender, and even if,', metadata={}), Document(page_content='When we consider how much greater would be our advantage in defending the air above this Island', metadata={}), Document(page_content='front, now on that, fighting on three fronts at once, battles fought by two or three divisions', metadata={})]


In [ ]:
for txt in result:
  print(txt.page_content)

on the beaches, we shall fight on the landing grounds, we shall fight in the fields and in the
fields and in the streets, we shall fight in the hills; we shall never surrender, and even if,
When we consider how much greater would be our advantage in defending the air above this Island
front, now on that, fighting on three fronts at once, battles fought by two or three divisions


In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

In [ ]:
llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs = {'k': 3})

In [ ]:
chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever= retriever)

In [ ]:
query = 'Where should we fight?'
answer = chain.run(query)
print(answer)

We shall fight on the beaches, in the landing grounds, in the fields, in the streets, and in the hills.
